# Henry PI 2: Machine Learning

• Caros incluyen el promedio

• robustscaler lidia mejor con outliers que standardscaler

## ------------- D A T A --- E X P L O R A T I O N --------------

In [1]:
# Importamos la librerías que necesitaremos

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [2]:
# Importamos el dataset con los datos de entrenamiento a un DataFrame de Pandas

df_original = pd.read_csv('datasets/properties_colombia_train.csv')
df_original.sample(5)

,Unnamed: 0,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,...,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type,geometry
117133,341112,L3R5xITE8JPkySHKyxdPXw==,Propiedad,2020-10-19,2021-01-18,2020-10-19,1.220589,-77.270066,Colombia,Nariño,...,NaN,NaN,190000000.0,COP,NaN,SE VENDE CASA ESQUINERA DE 6 ALCOBAS B/LA ESPE...,LA CASA ES ESQUINERA ESTA UBICADA EN EL B/ LA ...,Casa,Venta,POINT (-77.270066 1.220589)
38458,260733,/8XhDrGo/mSw6Dmg+Fha6w==,Propiedad,2020-10-21,2021-01-19,2020-10-21,NaN,NaN,Colombia,Meta,...,NaN,NaN,250000000.0,COP,NaN,ESPECTACULAR CASA RENTABLE DE DOS PISOS,ESPECTACULAR CASA RENTABLE DE DOS PISOS SECTOR...,Casa,Venta,POINT EMPTY
73306,850093,7gKL2KJ53nEXf2J+Pu8TPw==,Propiedad,2020-09-04,2020-09-10,2020-09-04,4.548942,-74.535235,Colombia,Cundinamarca,...,NaN,NaN,290000000.0,COP,NaN,vendo Lote en conjunto cerrado Anapoima,"Construya su vivienda campestre, como siempre ...",Lote,Venta,POINT (-74.535235 4.548942)
51508,636104,GiN7YVFhIk9e/LqLnDLaQg==,Propiedad,2020-07-29,2020-07-30,2020-07-29,6.247110,-75.604992,Colombia,Antioquia,...,NaN,NaN,660000000.0,COP,NaN,CASA EN VENTA SIMON BOLIVAR _ wasi1215905,Casa en venta ubicada en el sector de Simon Bo...,Casa,Venta,POINT (-75.60499191 6.24711018)
7617,642325,cBdqTVWFo3ye4eIjRVz7DA==,Propiedad,2020-08-15,2020-08-17,2020-08-15,NaN,NaN,Colombia,Antioquia,...,NaN,NaN,360000000.0,COP,NaN,Apartamento en Venta Ubicado en RIONEGRO,Codigo Inmueble 468 Apartamento Rionegro parqu...,Apartamento,Venta,POINT EMPTY


In [3]:
# Obtenemos algunos datos básicos del dataframe recién creado y el valor promedio de la columna que usaremos para crear la columna target

original_price_mean = df_original.price.mean()

print(f'• Original shape: {df_original.shape}\n')
print(f'• Original columns: {df_original.columns}\n')
print(f"• Original price column's mean: {original_price_mean}")

• Original shape: (197549, 27)

• Original columns: Index(['Unnamed: 0', 'id', 'ad_type', 'start_date', 'end_date', 'created_on',
       'lat', 'lon', 'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms', 'bedrooms',
       'bathrooms', 'surface_total', 'surface_covered', 'price', 'currency',
       'price_period', 'title', 'description', 'property_type',
       'operation_type', 'geometry'],
      dtype='object')

• Original price column's mean: 643605091.0064613


In [4]:
# Buscamos registros duplicados (no hay)

df_original.duplicated().value_counts()

False    197549
dtype: int64

In [5]:
# Buscamos valores nulos por columna (hay muchos, en particular en las columnas l4, l5, l6, rooms, bedrooms, surface_total, surface_covered y price_period)

df_original.isnull().sum()

Unnamed: 0              0
id                      0
ad_type                 0
start_date              0
end_date                0
created_on              0
lat                 49498
lon                 49498
l1                      0
l2                      0
l3                  11032
l4                 152182
l5                 170140
l6                 190682
rooms              170012
bedrooms           157024
bathrooms           41082
surface_total      190575
surface_covered    187747
price                  63
currency               67
price_period       161578
title                   1
description           121
property_type           0
operation_type          0
geometry                0
dtype: int64

In [6]:
print('\n\nDifferent values per column:\n\n')
for x in df_original:
    print(f'\n• {x}:\t{len(df_original[x].value_counts())}')



Different values per column:



• Unnamed: 0:	197549

• id:	197549

• ad_type:	1

• start_date:	145

• end_date:	446

• created_on:	145

• lat:	51077

• lon:	50108

• l1:	1

• l2:	31

• l3:	293

• l4:	58

• l5:	20

• l6:	146

• rooms:	29

• bedrooms:	37

• bathrooms:	20

• surface_total:	1035

• surface_covered:	783

• price:	6096

• currency:	2

• price_period:	1

• title:	95025

• description:	111371

• property_type:	8

• operation_type:	1

• geometry:	62807


From the output above we can see that there are several columns with only one value among the 197549 registers (ad_type, l1, price_period, operation_type). Also we can see that the 'Unnamed: 0' and the 'id' columns have unique values for each row (are identifiers) and thus are redundant.

We procceed to create another dataframe from the original one ignoring these columns.

In [7]:
# We create a new DataFrame which we will use to train our model with

df_train = df_original.drop(['ad_type', 'l1', 'price_period', 'operation_type', 'Unnamed: 0', 'id'], axis=1)
print(f'• Training DataFrame Shape: {df_train.shape}\n')
print(f'• Training DataFrame Columns: {df_train.columns}\n')
print(f'• Training DataFrame Price column mean: {df_train.price.mean()}')

• Training DataFrame Shape: (197549, 21)

• Training DataFrame Columns: Index(['start_date', 'end_date', 'created_on', 'lat', 'lon', 'l2', 'l3', 'l4',
       'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms', 'surface_total',
       'surface_covered', 'price', 'currency', 'title', 'description',
       'property_type', 'geometry'],
      dtype='object')

• Training DataFrame Price column mean: 643605091.0064613


In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197549 entries, 0 to 197548
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   start_date       197549 non-null  object 
 1   end_date         197549 non-null  object 
 2   created_on       197549 non-null  object 
 3   lat              148051 non-null  float64
 4   lon              148051 non-null  float64
 5   l2               197549 non-null  object 
 6   l3               186517 non-null  object 
 7   l4               45367 non-null   object 
 8   l5               27409 non-null   object 
 9   l6               6867 non-null    object 
 10  rooms            27537 non-null   float64
 11  bedrooms         40525 non-null   float64
 12  bathrooms        156467 non-null  float64
 13  surface_total    6974 non-null    float64
 14  surface_covered  9802 non-null    float64
 15  price            197486 non-null  float64
 16  currency         197482 non-null  obje

In [9]:
df_train.isnull().sum()

start_date              0
end_date                0
created_on              0
lat                 49498
lon                 49498
l2                      0
l3                  11032
l4                 152182
l5                 170140
l6                 190682
rooms              170012
bedrooms           157024
bathrooms           41082
surface_total      190575
surface_covered    187747
price                  63
currency               67
title                   1
description           121
property_type           0
geometry                0
dtype: int64

We can see that we have missing values in the 'price' column, the one we will be using to create our categorical target column.

In [10]:
df_train.dropna(subset=['price'], inplace=True)
price_mean_after_dropna = df_train.price.mean()

print(f'• Training DataFrame Shape (after dropna): {df_train.shape}\n')
print(f'• Training DataFrame Price column mean (after dropna): {df_train.price.mean()}')

print(f'\nPrice mean is the same before and after dropping missing values: {original_price_mean == price_mean_after_dropna}')

• Training DataFrame Shape (after dropna): (197486, 21)

• Training DataFrame Price column mean (after dropna): 643605091.0064613
Price mean is the same before and after dropping missing values: True


In [11]:
df_train.duplicated().value_counts()

False    193718
True       3768
dtype: int64

We can see that after dropping the redundant and identifier columns we got a total of 3768 duplicated registers.

We procceed to eliminate them.

In [12]:
df_train.drop_duplicates(inplace=True)
print(f'• Training DataFrame Shape (after drop_duplicates): {df_train.shape}\n')

• Training DataFrame Shape (after drop_duplicates): (193718, 21)



Now we will create the target column from the values of the 'price' column.

In [ ]:
df_2.price.min(), df_2.price.max()

In [ ]:
df_2.sort_values(by='price',ascending=False).head()

In [ ]:
df_2.price.value_counts()[0], df_2.price.value_counts()[345000000000.0]

In [ ]:
# Aquí podemos ver que tenemos 8 registros cuyo precio se encuentra en USD
df_2.currency.value_counts()

In [ ]:
df_2['Unnamed: 0'].duplicated().value_counts()

In [ ]:
df_2.loc('currency')

In [ ]:
df_2.price_period.value_counts()


In [ ]:
df_2.property_type.value_counts()


In [ ]:
df_2.operation_type.value_counts()


In [ ]:
df[['l1','l2','l3','l4','l5','l6']].sample(20)

In [ ]:
df_2.l1.value_counts()

In [ ]:
df_2.drop('l1', axis=1, inplace=True)

In [ ]:
df_2.l2.value_counts()

In [ ]:
df_2.l3.value_counts()

In [ ]:
df_2.l4.value_counts()